In [1]:
#数据库连接,必要的包
from pigeon import encryptAndDecrypt, connectOracle
import pandas as pd
import numpy as np
import datetime
handle_file = encryptAndDecrypt.encryptAndDecrypt()
dbInformation = handle_file.decrypt_file(dbName='wind')
connDB = connectOracle.connectOracle(dbInformation)

In [11]:
#根据sql拿到dataframe的数据
def gettable(sql,conn):
    db,cursor = conn.connect()
    cursor.execute(sql)
    result = cursor.fetchall() 
    des = cursor.description
    conn.close()
    col = [x[0] for x in des]
    df = pd.DataFrame(result,columns = col).replace(0,np.nan).dropna(how='all',axis=1)
    flag = [x[-4:] =='1231' for x in df.REPORT_PERIOD]
    flag[0] = True
    df = df[flag]
    df = df[:4]
    df.index = df.REPORT_PERIOD.values
    df.drop('REPORT_PERIOD',axis = 1,inplace = True)
    return df.dropna(how='all',axis=1).T.fillna(0)

In [10]:
#计算窄口径有息负债
compname = '杭州市交通投资集团有限公司'
decimal = 100000000
def getratio(compname,conn):
    now = datetime.datetime.now()
    time = str((now.year-5)*10000)
    sql = '''
    select s1.REPORT_PERIOD,ST_BORROW,NON_CUR_LIAB_DUE_WITHIN_1Y,LT_BORROW,BONDS_PAYABLE,LT_PAYABLE_TOT,LESS_FIN_EXP
    from (select REPORT_PERIOD,ST_BORROW,NON_CUR_LIAB_DUE_WITHIN_1Y,LT_BORROW,BONDS_PAYABLE,LT_PAYABLE_TOT
    from wind_user.CBondBalanceSheet where STATEMENT_TYPE = '408001000' and report_period>%s
    and s_info_compcode =  
    (select distinct s_info_compcode from wind_user.cbondissuer where s_info_compname = '%s')) s1
    inner join (select REPORT_PERIOD,LESS_FIN_EXP
    from wind_user.CBondIncome where STATEMENT_TYPE = '408001000' and report_period>%s
    and s_info_compcode =  
    (select distinct s_info_compcode from wind_user.cbondissuer where s_info_compname = '%s')) s2
    on s1.REPORT_PERIOD = s2.REPORT_PERIOD
    order by REPORT_PERIOD DESC
    '''%(time,compname,time,compname)
    
    db,cursor = conn.connect()
    cursor.execute(sql)
    result = cursor.fetchall() 
    des = cursor.description
    conn.close()
    col = [x[0] for x in des]
    df = pd.DataFrame(result,columns = col).replace(np.nan,0)
    ratio = df['LESS_FIN_EXP']/df[['ST_BORROW','NON_CUR_LIAB_DUE_WITHIN_1Y','LT_BORROW','BONDS_PAYABLE','LT_PAYABLE_TOT']].sum(axis = 1)
    ratio.index = df.REPORT_PERIOD
    flag = [x[-4:] =='1231' for x in df.REPORT_PERIOD]
    flag[0] = True
    ratio = ratio[flag]
    ratio = ratio[:4]
    ratio.columns = [compname]
    return ratio.to_dict()



In [56]:
ratio = getratio(compname,connDB)


In [59]:
ratio.to_dict()

{'20190930': 0.011181914103872303,
 '20181231': 0.019258216984206204,
 '20171231': 0.03193685969896259,
 '20161231': 0.03398176862754233}

In [8]:
def getintrat(compname,conn):
    sql = '''
    select s1.s_info_windcode,TRADE_DT,B_ANAL_PTMYEAR,B_ANAL_YTM,B_ISSUE_AMOUNTACT from
    (select s_info_windcode,TRADE_DT,B_ANAL_PTMYEAR,B_ANAL_YTM from wind_user.CBondValuation 
    where s_info_windcode in (select s_info_windcode from wind_user.cbondissuer where 
    s_info_compname = '%s') and TRADE_DT>20200000) s1
    inner join(select B_ISSUE_AMOUNTACT,s_info_windcode from wind_user.CBondDescription) s2
    on s1.s_info_windcode = s2.s_info_windcode
    order by TRADE_DT DESC
    '''%compname
    db,cursor = conn.connect()
    cursor.execute(sql)
    result = cursor.fetchall()
    des = cursor.description
    col = [x[0] for x in des]
    df = pd.DataFrame(result,columns = col).replace(np.nan,0).drop_duplicates(subset = ['S_INFO_WINDCODE'])
    a = df.B_ANAL_PTMYEAR
    a = (a-a.min())/(a.max()-a.min())
    wei = np.exp(-a)*df.B_ISSUE_AMOUNTACT
    result = (wei*df.B_ANAL_YTM).sum()/(wei.sum())
    return result

In [9]:
def getresult(compname,conn):
    print(compname+'begin')
    try:
        dic = getratio(compname,conn)
        dic['tipicalintrate'] = getintrat(compname,conn)
    except:
        print(compname+'not exist')
        return None
    return zip(compname,dic)

In [113]:
def getratiolist(complist,conn):
    now = datetime.datetime.now()
    time = str((now.year-5)*10000)
    b = np.ceil(len(complist)/1000).astype(int)
    lenth = np.ceil(len(complist)/b).astype(int)
    ans = []
    db,cursor = conn.connect()
    for i in range(b):
        tem = complist[i*lenth:i*lenth+lenth]
        compname = '\',\''.join(tem)

        sql = '''
        select s3.s_info_compname,s1.REPORT_PERIOD,ST_BORROW,NON_CUR_LIAB_DUE_WITHIN_1Y,LT_BORROW,BONDS_PAYABLE,LT_PAYABLE_TOT,LESS_FIN_EXP
        from (select distinct s_info_compname,s_info_compcode from wind_user.cbondissuer where s_info_compname in ('%s')) s3
        inner join 
        (select s_info_compcode,REPORT_PERIOD,ST_BORROW,NON_CUR_LIAB_DUE_WITHIN_1Y,LT_BORROW,BONDS_PAYABLE,LT_PAYABLE_TOT
        from wind_user.CBondBalanceSheet where STATEMENT_TYPE = '408001000' and report_period>%s) s1
        on s1.s_info_compcode = s3.s_info_compcode
        inner join (select s_info_compcode,REPORT_PERIOD,LESS_FIN_EXP
        from wind_user.CBondIncome where STATEMENT_TYPE = '408001000' and report_period>%s) s2
        on s1.REPORT_PERIOD = s2.REPORT_PERIOD and s1.s_info_compcode = s2.s_info_compcode
        order by REPORT_PERIOD DESC
        '''%(compname,time,time)
        cursor.execute(sql)
        result = cursor.fetchall()     
        des = cursor.description
        col = [x[0] for x in des]
        df = pd.DataFrame(result,columns = col).replace(np.nan,0).drop_duplicates(subset = ['S_INFO_COMPNAME','REPORT_PERIOD'])
        df['ratio'] = df['LESS_FIN_EXP']/df[['ST_BORROW','NON_CUR_LIAB_DUE_WITHIN_1Y','LT_BORROW','BONDS_PAYABLE','LT_PAYABLE_TOT']].sum(axis = 1)
        ratio = df.pivot(index = 'REPORT_PERIOD',columns = 'S_INFO_COMPNAME',values ='ratio')
        flag = [x[-4:] =='1231' for x in ratio.index]
        if flag[-1]==1:
            flag[-2] = True
            ratio = ratio[flag][-5:]
        else:
            flag[-1] = True
            ratio = ratio[flag][-4:]
        ans.append(ratio)
    conn.close()
    return pd.concat(ans,axis = 1)

        

In [114]:
def getintDic(complist,conn):
    now = datetime.datetime.now()
    time = now-datetime.timedelta(days=90)
    time = time.strftime('%Y%m%d')
    b = np.ceil(len(complist)/1000).astype(int)
    lenth = np.ceil(len(complist)/b).astype(int)
    intlist = {}
    db,cursor = conn.connect()
    for i in range(b):
        tem = complist[i*lenth:i*lenth+lenth]
        compname = '\',\''.join(tem)
        sql = '''
        select s3.s_info_compname,s1.s_info_windcode,TRADE_DT,B_ANAL_PTMYEAR,B_ANAL_YTM,B_ISSUE_AMOUNTACT 
        from (select distinct s_info_windcode,s_info_compname from wind_user.cbondissuer where s_info_compname in ('%s')) s3
        inner join (select s_info_windcode,TRADE_DT,B_ANAL_PTMYEAR,B_ANAL_YTM from wind_user.CBondValuation 
        where TRADE_DT>%s) s1
        on s1.s_info_windcode = s3.s_info_windcode
        inner join (select B_ISSUE_AMOUNTACT,s_info_windcode from wind_user.CBondDescription) s2
        on s1.s_info_windcode = s2.s_info_windcode
        order by TRADE_DT DESC
        '''%(compname,time)
        cursor.execute(sql)
        result = cursor.fetchall()
        print('result got')
        des = cursor.description
        col = [x[0] for x in des]
        df = pd.DataFrame(result,columns = col).drop_duplicates(subset = ['S_INFO_WINDCODE'])
        for item in df.groupby('S_INFO_COMPNAME'):
            name = item[0]
            temdf = item[1].replace(np.nan,0)
            if len(temdf)==1:
                intlist[name] = temdf.B_ANAL_PTMYEAR.values[0]
                continue
            a = temdf.B_ANAL_PTMYEAR
            a = (a-a.min())/(a.max()-a.min())
            wei = np.exp(-a)*temdf.B_ISSUE_AMOUNTACT
            result = (wei*temdf.B_ANAL_YTM).sum()/(wei.sum())
            intlist[name] = result
    return intlist

In [116]:

def getresult2(dftem,conn,name):
    complist = list(dftem.主体全称.values)
    print('satrt ratiolist')
    ans = getratiolist(complist,conn)
    print('satrt intdic')
    intdic = getintDic(complist,conn)
    series = pd.Series(intdic)
    df0 = pd.concat([dftem,ans.T,series],axis = 1,sort=False)
    col = df0.columns.values
    col[-1] = '发债代表利率'
    df0.columns = col
    df0.to_excel(name+'.xlsx')
    return df0


In [ ]:
bondissuer = pd.read_excel('bondissuer.xlsx')
rankfirst = pd.read_excel('rankfirst.xlsx')

In [117]:
rank = getresult2(rankfirst,connDB,'rankfirst2')

satrt ratiolist
satrt intdic
result got


/opt/software/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


result got


/opt/software/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/range.py:465: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  return self._int64index.union(other)
/opt/software/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:69: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)
